# Encoder-Decoder MT Attention Network

These are all the papers I reference:


1.   [Google Paper](https://arxiv.org/pdf/1609.08144.pdf): This was a study from google on how to configure encoder/decoder networks with attention for machine translation. Most of the decisions I made on what layers to use, how many to use, what attention to use, and other hyper parameters I use are determined from this. I mention most of them as they happen.
2.   [Luong et al](https://arxiv.org/pdf/1508.04025.pdf): This is the general scafolding I used for writing the network. This paper is a more digestable than the Bahdanau paper for the general strucutre of writing the seq2seq model.
3.   [Bahdanau et al](https://arxiv.org/pdf/1409.0473.pdf): Google referecnes 
Bahdanau attention as the better decision for attention. Bahdanau attention is referenced in the Luong paper too, but is slightly modified. The difference between the two is explored below. 



In [350]:
import numpy as np
import seaborn as sns
import tensorflow as tf

import re 
import os
from datetime import datetime

from nltk.translate.bleu_score import corpus_bleu

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Declare Static Variables

In [351]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


These parameters are mostly stolen from the Google Paper

In [352]:
EMBEDDING_DIM = 256
ATTENTION_UNITS = 512
ENCODER_UNITS = 1024
DECODER_UNITS = 1024
BATCH_SIZE = 64
MAX_INPUT_LENGTH = 32

### Load Data

In [353]:
BASE_PATH = '/content/drive/MyDrive/Data/Data'  # on local is path to directory

FORMAL_PATH_TRAIN = '{}/Supervised Data/Entertainment_Music/S_Formal_EM_Train.txt'.format(BASE_PATH)
INFORMAL_PATH_TRAIN = '{}/Supervised Data/Entertainment_Music/S_Informal_EM_Train.txt'.format(BASE_PATH)

FORMAL_PATH_HOLDOUT = '{}/Supervised Data/Entertainment_Music/S_Formal_EM_ValTest.txt'.format(BASE_PATH)
INFORMAL_PATH_HOLDOUT = '{}/Supervised Data/Entertainment_Music/S_Informal_EM_ValTest.txt'.format(BASE_PATH)

In [354]:
formal = open(FORMAL_PATH_TRAIN).read()
informal = open(INFORMAL_PATH_TRAIN).read()

formal_holdout = open(FORMAL_PATH_HOLDOUT).read()
informal_holdout = open(INFORMAL_PATH_HOLDOUT).read()

In [355]:
def process_sequence(seq):
    """This inserts a space in between the last word and a period"""
    s = re.sub('([.,!?()])', r' \1 ', seq)
    s = re.sub('\s{2,}', ' ', s)
    
    return '<start> ' + s + ' <end>'

In [356]:
f_corpus = [process_sequence(seq) for seq in formal.split('\n')]
if_corpus = [process_sequence(seq) for seq in informal.split('\n')]

f_holdout = [process_sequence(seq) for seq in formal_holdout.split('\n')]
if_holdout = [process_sequence(seq) for seq in informal_holdout.split('\n')]

### Preprocess data

In [357]:
def tokenize(corpus, tokenizer=None, maxlen=None):
    """ Tokenize data and pad sequences """
    if not tokenizer: 
        tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', 
                              oov_token='<OOV>')
        tokenizer.fit_on_texts(corpus)
    
    seqs = tokenizer.texts_to_sequences(corpus)
    padded_seqs = pad_sequences(seqs, padding='post', maxlen=maxlen)

    return padded_seqs, tokenizer

In [358]:
input_train, input_tokenizer = tokenize(if_corpus)
target_train, target_tokenizer = tokenize(f_corpus)

In [359]:
input_test, _ = tokenize(if_holdout, input_tokenizer, 32)
target_test, _ = tokenize(f_holdout, target_tokenizer)

In [360]:
buffer_size = len(input_train)
steps_per_epoch = len(input_train) // BATCH_SIZE
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

train = tf.data.Dataset.from_tensor_slices((input_train, target_train)).shuffle(buffer_size)
train = train.batch(BATCH_SIZE, drop_remainder=True)

test = tf.data.Dataset.from_tensor_slices((input_test, target_test)).batch(1)

In [361]:
example_input_batch, example_target_batch = next(iter(train))

In [362]:
test_in_batch, test_out_batch = next(iter(test))

### Encoder

Using two bidirectional LSTMs because that was reported to get best performance form the google paper. 

In [363]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm_1 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(
                self.encoder_units,
                return_sequences=True
            )
        )
        self.lstm_2 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(
                self.encoder_units,
                return_sequences=True,
                return_state=True
            )
        )

    def call(self, x):
        """Shove into latent space"""
        # x shape: (batch_size x max_length x embedding_dim)
        x = self.embedding(x)

        # x shape: (batch_size x max_length x 2 * encoder_units)
        x = self.lstm_1(x)

        # output shape: (batch_size x max_length x 2 * encoder_units)
        # h_f, h_b shapes: (batch_size x encoder_units)
        output, h_f, _, h_b, _ = self.lstm_2(x)

        return output, h_f, h_b


In [364]:
encoder_debug = Encoder(input_vocab_size, EMBEDDING_DIM, ENCODER_UNITS, BATCH_SIZE)

init_state = tf.zeros((BATCH_SIZE, ENCODER_UNITS))

sample_output, sample_forward_hidden, sample_backward_hidden = encoder_debug(example_input_batch)

### Attention Layer

In [365]:
class GlobalAttention(tf.keras.layers.Layer):
    """
    This is called GlobalAttention since that's what
    Bahdanau attention is called in the Luong paper
    and most of this implementation follows that 
    scaffolding. 

    The difference between this and global attention
    using concat 
    """
    def __init__(self, units):
        """These parameters follow notation from Bahdanau paper"""
        super(GlobalAttention, self).__init__()
        self.W = tf.keras.layers.Dense(units)
        self.U = tf.keras.layers.Dense(units)
        self.v = tf.keras.layers.Dense(1)

    def call(self, enc_opt, hidden_f, hidden_b):
        # concatenate hidden states as in eq 7 Bahdanau
        # hidden shape: (batch_size, 2 * lstm_units)
        hidden = tf.concat([hidden_f, hidden_b], axis=-1)

        # expand dims to meet shape of latent tensor
        # hidden_broad shape: (batch_size, 1, encoder_units * 2)
        hidden_broad = tf.expand_dims(hidden, 1)

        # Alignment model score from A.1.2 of Bahdanau et al 
        # score shape: (batch_size, max_length, v_units)
        score = self.v(tf.nn.tanh(self.W(hidden_broad) + self.U(enc_opt)))

        # softmax generalization of eq(7) Luong
        # attention_weights shape: (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)  

        # This takes weighted average with attention weights
        # context shape: (batch_size, 2 * encoder_units)
        context = attention_weights * enc_opt
        context = tf.reduce_sum(context, axis=1)

        return context, attention_weights

In [366]:
attention_layer = GlobalAttention(ATTENTION_UNITS)
attention_result, attention_weights = attention_layer(sample_output, 
                                                      sample_forward_hidden, 
                                                      sample_backward_hidden)

### Decoder

In [367]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, attention_units, decoder_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.attention = GlobalAttention(attention_units)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm_1 = tf.keras.layers.LSTM(decoder_units,
                                           return_sequences=True,
                                           return_state=True)
        self.flatten = tf.keras.layers.Flatten()
        self.opt = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden_f, hidden_b, encoder_output):
        context_vector, attention_weights = self.attention(encoder_output, hidden_f, hidden_b)

        # x shape: (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape: (batch_size, 1, embedding_dim + 2 * encoder_units)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # output shape: (batch_size, 1, decoder_units)
        # this shape is only important for expanding decoder depth
        output, h_f, c_f = self.lstm_1(x)

        # flatten to feed into opt
        # output shape: (batch_size, hidden_size)
        output = self.flatten(output)

        # get logits
        # x shape: (batch_size, vocab)
        x = self.opt(output)

        return x, h_f, attention_weights

In [368]:
# decoder = Decoder(target_vocab_size, EMBEDDING_DIM, ATTENTION_UNITS, DECODER_UNITS, BATCH_SIZE)

sample_decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

sample_decoder_output, _, _ = decoder(sample_decoder_input, sample_forward_hidden, sample_backward_hidden, sample_output)

### Optimizer and Loss Function

Here we define the optimizer and the loss function. In our loss function we mask the zeros since that's the padding.

Also of note is in the loss function. The reduction argument at default does some really wonky things which threw off all results. Had to change the reduciton to none, which at default is auto. Not exactly sure what it does in this context but it tries to sum over batches. I didn't work with it because I wanted to control all loss calculation manually. 

In [369]:
optimizer = tf.keras.optimizers.Adam()
static_loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [370]:
def loss_function(real, preds):
    """Calculate and return loss"""

    # caclulate loss
    loss = static_loss(real, preds)
    
    # create padding mask 
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss.dtype)
    
    # apply mask
    loss *= mask

    return tf.reduce_mean(loss)

### Training

This is the training loop. I'm using teacher forcing because I don't think I have enough computational power to use beam search. Google reccomends beam search with a beam width of 10, but that isn't an option here. Teacher forcing will provide better results than without using it. 

In [371]:
@tf.function
def train_step(inpt, trgt):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, dec_hidden_forward, dec_hidden_backward = encoder(inpt)

        # This feeds in the start token for the first initial state
        dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        for i in range(1, trgt.shape[1]):
            # dec_hidden shape: (batch_size, decoder_units)
            # dec_input shape: (batch_size, 1)
            predictions, dec_hidden_forward, _ = decoder(dec_input, 
                                                         dec_hidden_forward, 
                                                         dec_hidden_backward, 
                                                         enc_output)
            
            loss += loss_function(trgt[:, i], predictions)
            dec_input = tf.expand_dims(trgt[:, 1], 1)

        # Apply gradients 
        trainable_variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(gradients, trainable_variables))

        # return batch loss
        return loss

In [ ]:
EPOCHS = 10
encoder = Encoder(input_vocab_size, EMBEDDING_DIM, ENCODER_UNITS, BATCH_SIZE)
decoder = Decoder(target_vocab_size, EMBEDDING_DIM, ATTENTION_UNITS, 
                  DECODER_UNITS, BATCH_SIZE)

init_state = tf.zeros((BATCH_SIZE, 2 * ENCODER_UNITS))
for epoch in range(EPOCHS):
    start = datetime.now()

    total_loss = 0

    for inpt, trgt in train.take(steps_per_epoch):
        total_loss += train_step(inpt, trgt)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / BATCH_SIZE))
    print('Time taken {}\n'.format(datetime.now() - start))

In [ ]:
def evaluate(data):
    loss = 0
    results = []
    bleu = 0

    for i, (x, y) in enumerate(data):
        result = ''
        next_word = True

        # This feeds in the start token for the first initial state
        dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
        
        # Get inputs
        enc_output, dec_hidden_forward, dec_hidden_backward = encoder(x)

        i = 1  # iterative count
        while next_word: 
            # dec_hidden shape: (batch_size, decoder_units)
            # dec_input shape: (batch_size, 1)
            predictions, dec_hidden_forward, _ = decoder(dec_input, 
                                                         dec_hidden_forward, 
                                                         dec_hidden_backward, 
                                                         enc_output)
            loss += loss_function(y[:, i], predictions)
            dec_input = tf.expand_dims(predictions, 0)
            
            # max logit for tokenized word
            word_idx = tf.argmax(predictions[0]).numpy()
            word = target_tokenizer.index_word[word_idx]
            result += word + ' '

            if word == '<end>':
                next_word = False
            i += 1
        
        results.append(result)
        bleu += sentence_bleu(f_holdout[i], result)

    return results, loss, bleu / len(f_holdout)

In [ ]:
results, test_loss, bleu = evaluate(test)